# Agenda

1. What are decorators?
2. Decorating functions
3. Outer functions and decorators
4. Using the outer function in the decorator
5. Arguments to decorators
6. Decorating classes
7. Writing decorators as classes
8. Fixing some tiny decorator issues

In [2]:
def a():
    return 'A!'

def b():
    return 'B!'

print(a())
print(b())

A!
B!


In [3]:
# put lines above and below each printout

lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# About functions

1. Functions are objects, just like everything else in Python.
2. When I use `def`, I'm doing two things:
    - Creating a function object
    - Assigning that function object to a variable
3. When I assign a varible in a function, then     

In [ ]:
# put lines above and below each printout

lines = '-' * 60 + '\n'

def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(a())
print(b())